In [ ]:
using Plots, SparseArrays, LinearAlgebra

## Exercise 2

In [ ]:
#
#  Solves the steady-state heat equation in a square with conductivity
#  c(x,y) = 1 + x^2 + y^2:
#
#     -d/dx( (1+x^2+y^2) du/dx ) - d/dy( (1+x^2+y^2) du/dy ) = f(x),   
#                                                       0 < x,y < 1
#     u(x,0) = u(x,1) = u(0,y) = u(1,y) = 0
#
#  Uses a centered finite difference method.

#  Set up grid.
function make_A(n)
    h = 1/n;
    N = (n-1)^2;

    #  Form block tridiagonal finite difference matrix A and right-hand side 
    #  vector b.

    A= sparse(zeros(N,N));
    b = ones(N);         # Use right-hand side vector of all 1's.

    #  Loop over grid points in y direction.
    for j=1:n-1,
        yj = j*h;
        yjph = yj+h/2;  
        yjmh = yj-h/2;

    #  Loop over grid points in x direction.
      for i=1:n-1,
        xi = i*h;
        xiph = xi+h/2;  ximh = xi-h/2;
        aiphj = 1 + xiph^2 + yj^2;
        aimhj = 1 + ximh^2 + yj^2;
        aijph = 1 + xi^2 + yjph^2;
        aijmh = 1 + xi^2 + yjmh^2;
        k = (j-1)*(n-1) + i;
            
        A[k,k] = aiphj+aimhj+aijph+aijmh;
        if i > 1 
            A[k,k-1] = -aimhj
        end
        if i < n-1
            A[k,k+1] = -aiphj
        end
        if j > 1
            A[k,k-(n-1)] = -aijmh
        end;
        if j < n-1
            A[k,k+(n-1)] = -aijph
        end
      end
    end
    A = (1/h^2)*A;   # Remember to multiply A by (1/h^2).

    return A, b
end

In [ ]:
## Jacobi
function jacobi_iter(A, b, u0, max_iter)
    # Defining M and N
    M = diag(A)
    N = Diagonal(M) - A
    
    # Initializing storage for u
    u_iter = Vector{Vector{Float64}}(undef, max_iter+1)
    u_iter[1] = u0 
        
    for iter in 2:(max_iter+1)
        u_iter[iter] = (1 ./ M) .* (N*u_iter[iter-1] + b)
    end
   return u_iter
end

In [ ]:
## Gauss Seidel
function GS_iter(A, b, u0, max_iter)    
    # Defining M and N
    M = LowerTriangular(A)
    N = UpperTriangular(-A)
    N[diagind(N)] .= 0.0
    
    # Initializing storage for u
    u_iter = Vector{Vector{Float64}}(undef, max_iter+1)
    u_iter[1] = u0
    
    c = M \ b
    
    for iter in 2:(max_iter+1)
        u_iter[iter] = M \ (N*u_iter[iter-1]) + c
    end
   return u_iter
end

In [ ]:
## Successive overrelaxation
function SOR_iter(A, b, u0, ω, max_iter)    
    # Defining M and N
    L = LowerTriangular(-A)
    L[diagind(L)] .= 0.0
    U = UpperTriangular(-A)
    U[diagind(U)] .= 0.0
    D = Diagonal(A)
        
    M = (1/ω)*D - L
    N = ((1 - ω)/ω)*D + U
    
    # Initializing storage for u
    u_iter = Vector{Vector{Float64}}(undef, max_iter+1)
    u_iter[1] = u0
    
    c = M \ b
    
    for iter in 2:(max_iter+1)
        u_iter[iter] = M \ (N*u_iter[iter-1]) + c
    end
   return u_iter
end

In [ ]:
function make_residuals(A, b, u_iter)
    residuals = Vector{Float64}(undef, length(u_iter))
    for i in 1:length(u_iter)
        residuals[i] = norm(b - A*u_iter[i]) / norm(b)
    end
    return residuals
end

In [ ]:
# Constant for methods below
max_iter = 400
ω = 1.6
n = 25
u0 = rand((n-1)^2)
A, b = make_A(n);

In [ ]:
u_JAC = jacobi_iter(A, b, u0, max_iter);
u_GS = GS_iter(A, b, u0, max_iter);
u_SOR = SOR_iter(A, b, u0, 1.6, max_iter);

residuals_JAC = make_residuals(A, b, u_JAC);
residuals_GS = make_residuals(A, b, u_GS);
residuals_SOR = make_residuals(A, b, u_SOR);

In [ ]:
p = plot(yscale = :log10,
            xlabel = "Iterations",
            ylabel = "Relative residual norm",
            title = "Exercise 2: Residual")
p = plot!(residuals_JAC, 
            label = "Jacobi")
p = plot!(residuals_GS, 
            label = "GS")
p = plot!(residuals_SOR, 
            label = "SOR: ω = $ω")

In [ ]:
using IterativeSolvers

In [ ]:
p = plot(yscale = :log10,
            xlabel = "Iterations",
            ylabel = "Relative residual norm",
            title = "Exercise 2: CG")
p = plot!(residual_CG, 
            label = "CG");

In [ ]:
function ichol(B)
    A = copy(Matrix(B))
    N = size(A)[1] 
    L = zeros(N, N)
    for k in 1:N
        L[k,k] = sqrt(A[k,k])
        for i in (k+1):N
            if A[i,k] != 0
               A[i,k] = A[i,k] / A[k,k] 
            end
        end
        for j in (k+1):N
            for i in j:n
                if A[i,j] != 0
                   A[i,j] = A[i,j] - A[i, k]*A[j, k] 
                end
            end
        end
    end
    
    for i in 1:n
       for j in (i+1):n
            A[i,j] = 0
        end
    end
    return LowerTriangular(A)
end

In [ ]:
x, ch = cg(A, b, reltol = 1e-12, log=true)
residuals_CG = ch.data[:resnorm] ./ norm(b);

L = ichol(A)
x, ch = cg(A, b, reltol = 1e-12, log=true, Pl = L)
residual_CG_pre = ch.data[:resnorm] ./ norm(b);

In [ ]:
p = plot(yscale = :log10,
            xlabel = "Iterations",
            ylabel = "Relative residual norm",
            title = "Exercise 2: Residual")
p = plot!(residuals_JAC, 
            label = "Jacobi")
p = plot!(residuals_GS, 
            label = "GS")
p = plot!(residuals_SOR, 
            label = "SOR: ω = $ω")
p = plot!(residuals_CG, 
            label = "CG")
p = plot!(residual_CG_pre, 
            label = "CG_pre");

In [ ]:
function run_methods_fixed_h(N, max_iter = 400, ω = 1.6)

    u0 = rand((N-1)^2)
    A, b = make_A(N)

    u_JAC = jacobi_iter(A, b, u0, max_iter);
    u_GS = GS_iter(A, b, u0, max_iter);
    u_SOR = SOR_iter(A, b, u0, 1.6, max_iter);

    residuals_JAC = make_residuals(A, b, u_JAC);
    residuals_GS = make_residuals(A, b, u_GS);
    residuals_SOR = make_residuals(A, b, u_SOR);
    
    x, ch = cg(A, b, reltol = 1e-12, log=true)
    residuals_CG = ch.data[:resnorm] ./ norm(b);
    
    L = ichol(A)
    x, ch = cg(A, b, reltol = 1e-12, log=true, Pl = L)
    residual_CG_pre = ch.data[:resnorm] ./ norm(b);
    
    p = plot(yscale = :log10,
            xlabel = "Iterations",
            ylabel = "Relative residual norm",
            title = "Exercise 2: Residual. N = $N")
    p = plot!(residuals_JAC, 
                label = "Jacobi")
    p = plot!(residuals_GS, 
                label = "GS")
    p = plot!(residuals_SOR, 
                label = "SOR: ω = $ω")
    p = plot!(residuals_CG, 
                label = "CG")
    p = plot!(residual_CG_pre, 
                label = "CG_pre")
    return p
end

In [ ]:
exer_2_1 = run_methods_fixed_h(50, 1000, 1.3)

In [ ]:
savefig(exer_2_1, "../hw/figs/hw-5-exer-2-residual.png")

In [ ]:
function CG_method_ns(n_sequence, reltol = 1e-10)
    num_iters = []
    for n in n_sequence
        A, b = make_A(n)
        x, ch = cg(A, b, reltol = reltol, log=true)
        push!(num_iters, ch.iters)
    end
    return num_iters
end

In [ ]:
n_sequence = 2:50
reltol = 1e-12
num_iters_need = CG_method_ns(n_sequence, reltol);

In [ ]:
exer_2_2 = scatter(n_sequence, 
    num_iters_need,
    label = false,
    xlabel = "N",
    ylabel = "Number of iterations",
    title = "Exercise 2: Iterations needed to accuracy")

In [ ]:
savefig(exer_2_2, "../hw/figs/hw-5-exer-2-iterations-needed.png")